## XGBoost Hyperparameter Tuning (F1-Score)

In [7]:
# Load Libraries

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score

from xgboost import XGBClassifier

import optuna

In [8]:
# Load Data

df = pd.read_csv('bank_4.csv', index_col=0)

In [9]:
# Train / Test Split

X = df.drop(columns=['churn', 'complain', 'umap_1', 'umap_2'])
y = df['churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [13]:
# Hyperparameter tuning

def objective(trial):
    
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'gamma': trial.suggest_float('gamma', 0.0, 2.0),
        'max_depth': trial.suggest_int('max_depth', 2, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 20),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0)
    }
        
    model = XGBClassifier(
        **params,
        random_state=42,
        n_jobs=-1)
    
    threshold = 0.3
    
    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    scores = []
    
    for tr, te in skf.split(X_train, y_train):
        
        X_tr, X_te = X_train.iloc[tr], X_train.iloc[te]
        y_tr, y_te = y_train.iloc[tr], y_train.iloc[te]
        
        model.fit(X_tr, y_tr)
        prob = model.predict_proba(X_te)[:, 1]
        y_pred = np.where(prob < threshold, 0, 1)
        
        scores.append(f1_score(y_te, y_pred))
        
    return np.mean(scores)

In [20]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200, show_progress_bar=True)    # More trials because XGB is faster

[I 2024-07-10 13:43:39,459] A new study created in memory with name: no-name-a63212f8-1c1e-44b2-9499-5cd19ad2bdba


  0%|          | 0/200 [00:00<?, ?it/s]

[I 2024-07-10 13:43:44,730] Trial 0 finished with value: 0.5767396775751922 and parameters: {'n_estimators': 434, 'learning_rate': 0.26989994497105346, 'gamma': 1.9245903789370478, 'max_depth': 9, 'min_child_weight': 15, 'subsample': 0.6879059674399668, 'colsample_bytree': 0.8761076244992614}. Best is trial 0 with value: 0.5767396775751922.
[I 2024-07-10 13:43:46,503] Trial 1 finished with value: 0.6062728807128727 and parameters: {'n_estimators': 159, 'learning_rate': 0.13277673322758415, 'gamma': 1.0008541453958475, 'max_depth': 5, 'min_child_weight': 17, 'subsample': 0.8115777263684745, 'colsample_bytree': 0.9683952273530549}. Best is trial 1 with value: 0.6062728807128727.
[I 2024-07-10 13:43:48,066] Trial 2 finished with value: 0.61117045625842 and parameters: {'n_estimators': 183, 'learning_rate': 0.14686574021348206, 'gamma': 1.810041566537986, 'max_depth': 5, 'min_child_weight': 11, 'subsample': 0.9275363980355718, 'colsample_bytree': 0.712634023432369}. Best is trial 2 with va

In [22]:
print("Best trial:", study.best_trial)
print("Best hyperparameters:", study.best_params)

Best trial: FrozenTrial(number=198, state=1, values=[0.6274132174736339], datetime_start=datetime.datetime(2024, 7, 10, 14, 11, 13, 671369), datetime_complete=datetime.datetime(2024, 7, 10, 14, 11, 23, 451980), params={'n_estimators': 718, 'learning_rate': 0.015312996394551729, 'gamma': 1.9565233918410676, 'max_depth': 7, 'min_child_weight': 14, 'subsample': 0.824719070849096, 'colsample_bytree': 0.5000536641438886}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=1000, log=False, low=100, step=1), 'learning_rate': FloatDistribution(high=0.3, log=False, low=0.01, step=None), 'gamma': FloatDistribution(high=2.0, log=False, low=0.0, step=None), 'max_depth': IntDistribution(high=10, log=False, low=2, step=1), 'min_child_weight': IntDistribution(high=20, log=False, low=1, step=1), 'subsample': FloatDistribution(high=1.0, log=False, low=0.5, step=None), 'colsample_bytree': FloatDistribution(high=1.0, log=False, low=0.5, step=None)}